In [ ]:
# Import Tensorflow & Pathlib librairies
import tensorflow as tf 
import pathlib 
import pandas as pd 
import os
import io
import warnings
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding, Dense, LSTM, GRU, Dropout, BatchNormalization
#warnings.filterwarnings('ignore')

In [ ]:
# Import dataset with Pandas 
dataset = pd.read_csv("spam.csv", encoding="ISO-8859-1")
dataset.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
dataset= dataset.drop(["Unnamed: 2","Unnamed: 3","Unnamed: 4"], axis=1)

In [ ]:
len(dataset)

5572

In [ ]:
dataset.rename(columns = {'v1':'type', 'v2': 'mail'}, inplace = True)

In [ ]:
dataset.head()

,type,mail
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
# check count and unique and top values and their frequency
dataset['type'].value_counts()

ham     4825
spam     747
Name: type, dtype: int64

In [ ]:
# target replaced by 1 or 0
dataset['type'] = dataset['type'].apply(lambda x:1 if x=='spam' else 0)

In [ ]:
!python -m spacy download en_core_web_md -q

2022-12-07 15:03:08.950023: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
     |████████████████████████████████| 42.8 MB 1.3 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:
# Import Spacy and french initialisation
import en_core_web_md
nlp = en_core_web_md.load()

In [ ]:
# Import Stop words 
from spacy.lang.en.stop_words import STOP_WORDS

In [ ]:
dataset["mail_clean"] = dataset["mail"].apply(lambda x:''.join(ch for ch in x if ch.isalnum() or ch==" " or ch=="'"))
dataset["mail_clean"] = dataset["mail_clean"].apply(lambda x: x.replace("å","").lower().strip())
dataset["mail_clean"] = dataset["mail_clean"].apply(lambda x: " ".join([token.lemma_ for token in nlp(x) if (token.lemma_ not in STOP_WORDS) and (token.text not in STOP_WORDS)]))

dataset

,type,mail,mail_clean
0,0,"Go until jurong point, crazy.. Available only ...",jurong point crazy available bugis n great wor...
1,0,Ok lar... Joking wif u oni...,ok lar joke wif u oni
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...
3,0,U dun say so early hor... U c already then say...,u dun early hor u c
4,0,"Nah I don't think he goes to usf, he lives aro...",nah think usf live
...,...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...,2nd time try 2 contact u u win 750 pound prize...
5568,0,Will Ì_ b going to esplanade fr home?,ì b esplanade fr home
5569,0,"Pity, * was in mood for that. So...any other s...",pity mood soany suggestion
5570,0,The guy did some bitching but I acted like i'd...,guy bitching act like interested buy week free


In [ ]:
mask = dataset["mail_clean"].isna()==False
dataset = dataset[mask]

In [ ]:
import numpy as np
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=1000) # instanciate the tokenizer
tokenizer.fit_on_texts(dataset["mail_clean"])
dataset["mail_encoded"] = tokenizer.texts_to_sequences(dataset.mail_clean)
dataset["len_mail"] = dataset["mail_encoded"].apply(lambda x: len(x))
dataset = dataset[dataset["len_mail"]!=0]

In [ ]:
mails_pad = tf.keras.preprocessing.sequence.pad_sequences(dataset.mail_encoded, padding="post")

In [ ]:
# Train Test Split
xtrain, xval, ytrain, yval = train_test_split(mails_pad,dataset.type, test_size=0.3)

In [ ]:
train = tf.data.Dataset.from_tensor_slices((xtrain, ytrain))
val = tf.data.Dataset.from_tensor_slices((xval, yval))

In [ ]:
train_batch = train.shuffle(len(train)).batch(64)
val_batch = val.shuffle(len(val)).batch(64)

In [ ]:
 # Regardons un batch 
for review, star in train_batch.take(1):
  print(review, star)

tf.Tensor(
[[ 51  58  19 ...   0   0   0]
 [426   0   0 ...   0   0   0]
 [ 71   4   1 ...   0   0   0]
 ...
 [272  39  47 ...   0   0   0]
 [  7   9  35 ...   0   0   0]
 [ 18  61 122 ...   0   0   0]], shape=(64, 54), dtype=int32) tf.Tensor(
[0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0], shape=(64,), dtype=int64)


In [ ]:
vocab_size = len(tokenizer.word_index)
model_lstm = tf.keras.Sequential([
                  Embedding(vocab_size+1, 64, input_shape=[review.shape[1],],name="embedding"),
                  LSTM(units=64, return_sequences=True), # maintains the sequential nature
                  Dropout(0.2),
                  LSTM(units=32, return_sequences=False), # returns the last output
                  Dense(16, activation='relu'),
                  Dense(8, activation='relu'),

                  Dense(1, activation="sigmoid", name="last")
])

In [ ]:
model_lstm.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 54, 64)            522944    
                                                                 
 lstm_6 (LSTM)               (None, 54, 64)            33024     
                                                                 
 dropout_5 (Dropout)         (None, 54, 64)            0         
                                                                 
 lstm_7 (LSTM)               (None, 32)                12416     
                                                                 
 dense_24 (Dense)            (None, 16)                528       
                                                                 
 dense_25 (Dense)            (None, 8)                 136       
                                                                 
 last (Dense)                (None, 1)                

In [ ]:
optimizer= tf.keras.optimizers.Adam()

model_lstm.compile(optimizer=optimizer,
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy()])

In [ ]:
model_lstm.fit(train_batch,
              epochs=30, 
              validation_data=val_batch)

Epoch 1/30
59/59 [==============================] - 11s 114ms/step - loss: 0.5148 - binary_accuracy: 0.8474 - val_loss: 0.3882 - val_binary_accuracy: 0.8696
Epoch 2/30
59/59 [==============================] - 6s 97ms/step - loss: 0.4091 - binary_accuracy: 0.8585 - val_loss: 0.3876 - val_binary_accuracy: 0.8696
Epoch 3/30
59/59 [==============================] - 6s 97ms/step - loss: 0.4092 - binary_accuracy: 0.8585 - val_loss: 0.3875 - val_binary_accuracy: 0.8696
Epoch 4/30
59/59 [==============================] - 6s 97ms/step - loss: 0.4080 - binary_accuracy: 0.8585 - val_loss: 0.3875 - val_binary_accuracy: 0.8696
Epoch 5/30
59/59 [==============================] - 6s 96ms/step - loss: 0.4080 - binary_accuracy: 0.8585 - val_loss: 0.3872 - val_binary_accuracy: 0.8696
Epoch 6/30
59/59 [==============================] - 6s 99ms/step - loss: 0.4084 - binary_accuracy: 0.8585 - val_loss: 0.3899 - val_binary_accuracy: 0.8696
Epoch 7/30
59/59 [==============================] - 6s 97ms/step - l

In [ ]:
#introduction de la class_weight, ayant en target 2 classes représentées de manière déséquilibrée
weights = 1/(dataset["type"]).value_counts()
weights = weights * len(dataset)/2
weights = {index : values for index , values in zip(weights.index,weights.values)}
weights

{0: 0.5801419965576592, 1: 3.619463087248322}

In [ ]:
# Model training 
model_lstm.fit(train_batch,
          epochs=30, 
          validation_data=val_batch,
          class_weight=weights)

Epoch 1/30
59/59 [==============================] - 9s 95ms/step - loss: 1.0369 - binary_accuracy: 0.8585 - val_loss: 0.3929 - val_binary_accuracy: 0.8696
Epoch 2/30
59/59 [==============================] - 6s 97ms/step - loss: 0.9755 - binary_accuracy: 0.8585 - val_loss: 0.4006 - val_binary_accuracy: 0.8696
Epoch 3/30
59/59 [==============================] - 6s 95ms/step - loss: 0.9313 - binary_accuracy: 0.8585 - val_loss: 0.4093 - val_binary_accuracy: 0.8696
Epoch 4/30
59/59 [==============================] - 6s 97ms/step - loss: 0.8976 - binary_accuracy: 0.8585 - val_loss: 0.4190 - val_binary_accuracy: 0.8696
Epoch 5/30
59/59 [==============================] - 6s 97ms/step - loss: 0.8700 - binary_accuracy: 0.8585 - val_loss: 0.4289 - val_binary_accuracy: 0.8696
Epoch 6/30
59/59 [==============================] - 6s 95ms/step - loss: 0.8471 - binary_accuracy: 0.8585 - val_loss: 0.4392 - val_binary_accuracy: 0.8696
Epoch 7/30
59/59 [==============================] - 6s 97ms/step - los

In [ ]:
import json
tf.keras.utils.get_file("/content/model_lstm_reg.h5",
                        origin="https://full-stack-assets.s3.eu-west-3.amazonaws.com/models/M08_Deep_learning/Text_classification/model_lstm_reg.h5")
tf.keras.utils.get_file("/content/LSTM_history_reg.json",
                        origin="https://full-stack-assets.s3.eu-west-3.amazonaws.com/models/M08_Deep_learning/Text_classification/LSTM_history_reg.json")
LSTM_history_reg = json.load(open("/content/LSTM_history_reg.json", 'r'))
model_lstm_reg = tf.keras.models.load_model("/content/model_lstm_reg.h5")

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(y=LSTM_history_reg["loss"],
                    mode='lines',
                    name='loss'))
fig.add_trace(go.Scatter(y=LSTM_history_reg["val_loss"],
                    mode='lines',
                    name='val_loss'))
fig.show()


In [ ]:
#LSTM ne donne pas du tout de bon resultat 
vocab_size = len(tokenizer.word_index)
model_gru = tf.keras.Sequential([
                  Embedding(vocab_size+1, 64, input_shape=[review.shape[1],],name="embedding"),
                  GRU(units=64, return_sequences=True), # maintains the sequential nature
                  Dropout(0.15),
                  GRU(units=32, return_sequences=False), # returns the last output
                  Dense(16, activation='relu'),
                  Dense(8, activation='relu'),
                  Dense(1, activation="sigmoid")
])

In [ ]:
optimizer= tf.keras.optimizers.Adam()

model_gru.compile(optimizer=optimizer,
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy()])

In [ ]:
model_gru.fit(train_batch,
              epochs=20, 
              validation_data=val_batch)

Epoch 1/20
59/59 [==============================] - 12s 112ms/step - loss: 0.5025 - binary_accuracy: 0.8469 - val_loss: 0.3887 - val_binary_accuracy: 0.8696
Epoch 2/20
59/59 [==============================] - 6s 96ms/step - loss: 0.4076 - binary_accuracy: 0.8585 - val_loss: 0.3890 - val_binary_accuracy: 0.8696
Epoch 3/20
59/59 [==============================] - 6s 95ms/step - loss: 0.3505 - binary_accuracy: 0.8800 - val_loss: 0.1428 - val_binary_accuracy: 0.9771
Epoch 4/20
59/59 [==============================] - 6s 95ms/step - loss: 0.0777 - binary_accuracy: 0.9833 - val_loss: 0.0748 - val_binary_accuracy: 0.9765
Epoch 5/20
59/59 [==============================] - 6s 96ms/step - loss: 0.0382 - binary_accuracy: 0.9921 - val_loss: 0.0881 - val_binary_accuracy: 0.9790
Epoch 6/20
59/59 [==============================] - 6s 96ms/step - loss: 0.0312 - binary_accuracy: 0.9918 - val_loss: 0.0863 - val_binary_accuracy: 0.9790
Epoch 7/20
59/59 [==============================] - 6s 95ms/step - l

In [ ]:
model_gru.save("model_gru.h5")
json.dump(model_gru.history.history, open("/content/GRU_history.json", 'w'))

In [ ]:
tf.keras.utils.get_file("/content/model_gru.h5",
                        origin="https://full-stack-assets.s3.eu-west-3.amazonaws.com/models/M08_Deep_learning/Text_classification/model_gru.h5")
tf.keras.utils.get_file("/content/GRU_history.json",
                        origin="https://full-stack-assets.s3.eu-west-3.amazonaws.com/models/M08_Deep_learning/Text_classification/GRU_history.json")
GRU_history = json.load(open("/content/GRU_history.json", 'r'))
model_gru = tf.keras.models.load_model("/content/model_gru.h5")

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=GRU_history["loss"],
                    mode='lines',
                    name='loss'))
fig.add_trace(go.Scatter(y=GRU_history["val_loss"],
                    mode='lines',
                    name='val_loss'))
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=GRU_history["binary_accuracy"],
                    mode='lines',
                    name='accuracy'))
fig.add_trace(go.Scatter(y=GRU_history["val_binary_accuracy"],
                    mode='lines',
                    name='val_accuracy'))
fig.show()